In [1]:
import pandas as pd
import numpy as np
import csv
from keras.utils import to_categorical

In [26]:
test = pd.read_csv('compdata\\kaggletest.csv')
train = pd.read_csv('compdata\\kaggletrain.csv')

from tensorflow.keras.utils import to_categorical
labels_train = train['label']
cat = to_categorical(labels_train)


trainingset = train.drop('label', axis=1)
trainingset = trainingset/255
trainingset = trainingset.to_numpy()
trainingset = trainingset.reshape(42000,28,28,1)

testset = test
testset = testset/255
testset = testset.to_numpy()
testset = testset.reshape(28000,28,28,1)


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from tensorflow.keras.callbacks import EarlyStopping



In [56]:
convModel = Sequential([
    Conv2D(filters=64,kernel_size=(4,4), input_shape=(28,28,1),padding='SAME'),
    MaxPool2D(pool_size=(4,4)),
    Conv2D(filters=128, kernel_size=(2,2),padding='SAME'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(42, activation='relu'),
    Dense(10,activation='softmax')
])
#Scaling Training Data 
convModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
convModel.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 28, 28, 64)        1088      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 7, 7, 64)          0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 7, 7, 128)         32896     
                                                                 
 flatten_7 (Flatten)         (None, 6272)              0         
                                                                 
 dense_12 (Dense)            (None, 64)                401472    
                                                                 
 dense_13 (Dense)            (None, 42)                2730      
                                                      

In [57]:
from tensorflow.keras.callbacks import EarlyStopping

In [58]:
callback = EarlyStopping(monitor='loss', patience=2)
convModel.fit(trainingset, cat, epochs=25,callbacks=[callback])

Epoch 1/25
1313/1313 [==============================] - 11s 8ms/step - loss: 0.1804 - accuracy: 0.9451
Epoch 2/25
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0661 - accuracy: 0.9793
Epoch 3/25
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0500 - accuracy: 0.9846
Epoch 4/25
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0436 - accuracy: 0.9862
Epoch 5/25
1313/1313 [==============================] - 10s 8ms/step - loss: 0.0371 - accuracy: 0.9882
Epoch 6/25
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0325 - accuracy: 0.9895
Epoch 7/25
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0285 - accuracy: 0.9909
Epoch 8/25
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0239 - accuracy: 0.9926
Epoch 9/25
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0272 - accuracy: 0.9913
Epoch 10/25
1313/1313 [==============================] - 10s 8ms/step - l

In [41]:
results = convModel.predict(testset)
maxeach = np.argmax(results,axis=1)

875/875 [==============================] - 2s 2ms/step


In [55]:
findata = pd.DataFrame(maxeach)
findata.columns = ["Label"]
findata.rename_axis('ImageId',inplace=True)
findata.index = range(1,len(findata)+1)
findata.to_csv("digitrecogpred.csv",index=True)